In [1]:
# YOLOv5 🚀 by Ultralytics, GPL-3.0 license
"""
Train a YOLOv5 model on a custom dataset

Usage:
    $ python path/to/train.py --data coco128.yaml --weights yolov5s.pt --img 640
"""
from label_converter import BACKGROUND_CLASS_ID
from timeit import default_timer as timer
from collections import defaultdict
import pdb
import argparse
from label_converter import qt2yolo_optimized, qt2yolo
from GPUtil import showUtilization as gpu_usage

from label_filter import filter_qt

from PIL.ImageFont import truetype
from label_converter import yolo2bcc_new, find_union_cstargets, targetize, yolo2bcc_newer
from train_with_bcc import convert_yolo2bcc, nn_predict, convert_cs_yolo2bcc
from train_with_bcc import read_crowdsourced_labels, init_bcc_params, \
    init_nn_output, compute_param_confusion_matrices, init_metrics, update_bcc_metrics
from lib.BCCNet.VariationalInference.VB_iteration_yolo import VB_iteration as VBi_yolo
import logging
import math
import os
import random
import sys
import time
from copy import deepcopy
from pathlib import Path

import numpy as np
import torch
import torch.distributed as dist
import torch.nn as nn
import yaml
from torch.cuda import amp
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.optim import Adam, SGD, lr_scheduler
from tqdm import tqdm

# FILE = Path(__file__).absolute()
# sys.path.append(FILE.parents[0].as_posix())  # add yolov5/ to path

import val  # for end-of-epoch mAP
from models.experimental import attempt_load
from models.yolo import Model
from utils.autoanchor import check_anchors
from utils.datasets import create_dataloader
from utils.general import labels_to_class_weights, increment_path, labels_to_image_weights, init_seeds, \
    strip_optimizer, get_latest_run, check_dataset, check_file, check_git_status, check_img_size, \
    check_requirements, print_mutation, set_logging, one_cycle, colorstr, methods
from utils.downloads import attempt_download
from utils.loss import ComputeLoss
from utils.plots import plot_labels, plot_evolve
from utils.torch_utils import EarlyStopping, ModelEMA, de_parallel, intersect_dicts, select_device, \
    torch_distributed_zero_first
from utils.loggers.wandb.wandb_utils import check_wandb_resume
from utils.metrics import fitness
from utils.loggers import Loggers
from utils.callbacks import Callbacks

LOGGER = logging.getLogger(__name__)
LOCAL_RANK = int(os.getenv('LOCAL_RANK', -1))  # https://pytorch.org/docs/stable/elastic/run.html
RANK = int(os.getenv('RANK', -1))
WORLD_SIZE = int(os.getenv('WORLD_SIZE', 1))

In [2]:
def parse_opt(known=False):
    parser = argparse.ArgumentParser()
    parser.add_argument('--bcc_epoch', type=int, default=0, help='start-epoch for BCC+YOLO run; use -1 for no BCC.')
    parser.add_argument('--qtfilter_epoch', type=int, default=-1, help='start-epoch for qt-filter; use -1 for no qt-filter.')
    parser.add_argument('--qt_thres_mode', type=str, default='', help="one of '', 'conf-count', 'entropy', 'conf-val'")
    parser.add_argument('--qt_thres', type=float, default=0.0, help="the threshold value.")
    parser.add_argument('--hybrid_entropy_thres', type=float, default=0.0, help="the entropy threshold value (only to be used when running the hybrid filter).")
    parser.add_argument('--hybrid_conf_thres', type=float, default=0.0, help="the confidence threshold value (only to be used when running the hybrid filter).")
    parser.add_argument('--weights', type=str, default='yolov5s.pt', help='initial weights path')
    parser.add_argument('--cfg', type=str, default='', help='model.yaml path')
    parser.add_argument('--data', type=str, default='data/coco128.yaml', help='dataset.yaml path')
    parser.add_argument('--hyp', type=str, default='data/hyps/hyp.scratch.yaml', help='hyperparameters path')
    parser.add_argument('--epochs', type=int, default=300)
    parser.add_argument('--batch-size', type=int, default=16, help='total batch size for all GPUs')
    parser.add_argument('--imgsz', '--img', '--img-size', type=int, default=640, help='train, val image size (pixels)')
    parser.add_argument('--rect', action='store_true', help='rectangular training')
    parser.add_argument('--resume', nargs='?', const=True, default=False, help='resume most recent training')
    parser.add_argument('--nosave', action='store_true', help='only save final checkpoint')
    parser.add_argument('--noval', action='store_true', help='only validate final epoch')
    parser.add_argument('--noautoanchor', action='store_true', help='disable autoanchor check')
    parser.add_argument('--evolve', type=int, nargs='?', const=300, help='evolve hyperparameters for x generations')
    parser.add_argument('--bucket', type=str, default='', help='gsutil bucket')
    parser.add_argument('--cache', type=str, nargs='?', const='ram', help='--cache images in "ram" (default) or "disk"')
    parser.add_argument('--image-weights', action='store_true', help='use weighted image selection for training')
    parser.add_argument('--device', default='', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
    parser.add_argument('--multi-scale', action='store_true', help='vary img-size +/- 50%%')
    parser.add_argument('--single-cls', action='store_true', help='train multi-class data as single-class')
    parser.add_argument('--adam', action='store_true', help='use torch.optim.Adam() optimizer')
    parser.add_argument('--sync-bn', action='store_true', help='use SyncBatchNorm, only available in DDP mode')
    parser.add_argument('--workers', type=int, default=8, help='maximum number of dataloader workers')
    parser.add_argument('--project', default='runs/train', help='save to project/name')
    parser.add_argument('--entity', default=None, help='W&B entity')
    parser.add_argument('--name', default='exp', help='save to project/name')
    parser.add_argument('--exist-ok', action='store_true', help='existing project/name ok, do not increment')
    parser.add_argument('--quad', action='store_true', help='quad dataloader')
    parser.add_argument('--linear-lr', action='store_true', help='linear LR')
    parser.add_argument('--label-smoothing', type=float, default=0.0, help='Label smoothing epsilon')
    parser.add_argument('--upload_dataset', action='store_true', help='Upload dataset as W&B artifact table')
    parser.add_argument('--bbox_interval', type=int, default=-1, help='Set bounding-box image logging interval for W&B')
    parser.add_argument('--save_period', type=int, default=-1, help='Log model after every "save_period" epoch')
    parser.add_argument('--artifact_alias', type=str, default="latest", help='version of dataset artifact to be used')
    parser.add_argument('--local_rank', type=int, default=-1, help='DDP parameter, do not modify')
    parser.add_argument('--freeze', type=int, default=0, help='Number of layers to freeze. backbone=10, all=24')
    parser.add_argument('--patience', type=int, default=1100, help='EarlyStopping patience (epochs)')
    opt = parser.parse_known_args()[0] if known else parser.parse_args('')
    return opt

## Main

In [5]:
data_name = 'bcc'
opt = parse_opt()
opt.data = f'data/{data_name}.yaml' # Does not make any difference!
data_dict_path = f'../../datasets/{data_name}'
opt.exist_ok = False
opt.cache = None
opt.workers = 0
opt.batch_size = 20 # Change this to number of train images
opt.epochs = 5
opt.bcc_epoch = 0

# Checks
set_logging(RANK)
if RANK in [-1, 0]:
    print(colorstr('train: ') + ', '.join(f'{k}={v}' for k, v in vars(opt).items()))
#     check_git_status()
#     check_requirements(requirements=FILE.parent / 'requirements.txt', exclude=['thop'])

# Resume
if opt.resume and not check_wandb_resume(opt) and not opt.evolve:  # resume an interrupted run
    ckpt = opt.resume if isinstance(opt.resume, str) else get_latest_run()  # specified or most recent path
    assert os.path.isfile(ckpt), 'ERROR: --resume checkpoint does not exist'
    with open(Path(ckpt).parent.parent / 'opt.yaml') as f:
        opt = argparse.Namespace(**yaml.safe_load(f))  # replace
    opt.cfg, opt.weights, opt.resume = '', ckpt, True  # reinstate
    LOGGER.info(f'Resuming training from {ckpt}')
else:
    opt.data, opt.cfg, opt.hyp = check_file(opt.data), check_file(opt.cfg), check_file(opt.hyp)  # check files
    assert len(opt.cfg) or len(opt.weights), 'either --cfg or --weights must be specified'
    if opt.evolve:
        opt.project = 'runs/evolve'
        opt.exist_ok = opt.resume
    opt.save_dir = str(increment_path(Path(opt.project) / opt.name, exist_ok=opt.exist_ok))

# DDP mode
device = select_device(opt.device, batch_size=opt.batch_size)
if LOCAL_RANK != -1:
    from datetime import timedelta
    assert torch.cuda.device_count() > LOCAL_RANK, 'insufficient CUDA devices for DDP command'
    assert opt.batch_size % WORLD_SIZE == 0, '--batch-size must be multiple of CUDA device count'
    assert not opt.image_weights, '--image-weights argument is not compatible with DDP training'
    assert not opt.evolve, '--evolve argument is not compatible with DDP training'
    torch.cuda.set_device(LOCAL_RANK)
    device = torch.device('cuda', LOCAL_RANK)
    dist.init_process_group(backend="nccl" if dist.is_nccl_available() else "gloo")
hyp = opt.hyp
callbacks=Callbacks()

YOLOv5 🚀 fb40f39 torch 1.9.0 CPU



train: bcc_epoch=0, qtfilter_epoch=-1, qt_thres_mode=, qt_thres=0.0, hybrid_entropy_thres=0.0, hybrid_conf_thres=0.0, weights=yolov5s.pt, cfg=, data=data/bcc.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=5, batch_size=20, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=0, project=runs/train, entity=None, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, freeze=0, patience=1100


## Train

In [8]:
torchMode = True
save_dir, epochs, batch_size, weights, single_cls, evolve, data, cfg, resume, noval, nosave, workers, freeze, = \
    Path(opt.save_dir), opt.epochs, opt.batch_size, opt.weights, opt.single_cls, opt.evolve, opt.data, opt.cfg, \
    opt.resume, opt.noval, opt.nosave, opt.workers, opt.freeze

bcc_epoch = opt.bcc_epoch
qtfilter_epoch = opt.qtfilter_epoch
qt_thres_mode = opt.qt_thres_mode
qt_thres = opt.qt_thres
hybrid_entropy_thres = opt.hybrid_entropy_thres
hybrid_conf_thres = opt.hybrid_conf_thres
# if not opt.bcc:
#     bcc_epoch = -1
# Directories
w = save_dir / 'weights'  # weights dir
w.mkdir(parents=True, exist_ok=True)  # make dir
last, best = w / 'last.pt', w / 'best.pt'

# Hyperparameters
if isinstance(hyp, str):
    with open(hyp) as f:
        hyp = yaml.safe_load(f)  # load hyps dict
LOGGER.info(colorstr('hyperparameters: ') + ', '.join(f'{k}={v}' for k, v in hyp.items()))

# Save run settings
with open(save_dir / 'hyp.yaml', 'w') as f:
    yaml.safe_dump(hyp, f, sort_keys=False)
with open(save_dir / 'opt.yaml', 'w') as f:
    yaml.safe_dump(vars(opt), f, sort_keys=False)
data_dict = {
    'path': data_dict_path,  # dataset root dir
    'train': 'images/train',  # train images (relative to 'path') 128 images
    'val': 'images/val',  # val images (relative to 'path') 128 images
    'test': 'images/test', # test images (optional)
    'nc': 2,  # number of classes
    'names': ['bone-loss', 'dental-caries']  # class names
}
for x in ['train', 'val', 'test']:
    data_dict[x] = os.path.join(data_dict['path'], data_dict[x])

# Loggers
if RANK in [-1, 0]:
    loggers = Loggers(save_dir, weights, opt, hyp, LOGGER)  # loggers instance
    if loggers.wandb:
        data_dict = loggers.wandb.data_dict
        if resume:
            weights, epochs, hyp = opt.weights, opt.epochs, opt.hyp

    # Register actions
    for k in methods(loggers):
        callbacks.register_action(k, callback=getattr(loggers, k))

# Config
plots = not evolve  # create plots
cuda = device.type != 'cpu'
init_seeds(1 + RANK)
with torch_distributed_zero_first(RANK):
    data_dict = data_dict or check_dataset(data)  # check if None
train_path, val_path = data_dict['train'], data_dict['val']
nc = 1 if single_cls else int(data_dict['nc'])  # number of classes
names = ['item'] if single_cls and len(data_dict['names']) != 1 else data_dict['names']  # class names
assert len(names) == nc, f'{len(names)} names found for nc={nc} dataset in {data}'  # check
is_coco = data.endswith('coco.yaml') and nc == 80  # COCO dataset

# Model
pretrained = weights.endswith('.pt')
if pretrained:
    with torch_distributed_zero_first(RANK):
        weights = attempt_download(weights)  # download if not found locally
    ckpt = torch.load(weights, map_location=device)  # load checkpoint
    model = Model(cfg or ckpt['model'].yaml, ch=3, nc=nc, anchors=hyp.get('anchors')).to(device)  # create
    exclude = ['anchor'] if (cfg or hyp.get('anchors')) and not resume else []  # exclude keys
    csd = ckpt['model'].float().state_dict()  # checkpoint state_dict as FP32
    csd = intersect_dicts(csd, model.state_dict(), exclude=exclude)  # intersect
    model.load_state_dict(csd, strict=False)  # load
    LOGGER.info(f'Transferred {len(csd)}/{len(model.state_dict())} items from {weights}')  # report
else:
    model = Model(cfg, ch=3, nc=nc, anchors=hyp.get('anchors')).to(device)  # create

# Freeze
freeze = [f'model.{x}.' for x in range(freeze)]  # layers to freeze
for k, v in model.named_parameters():
    v.requires_grad = True  # train all layers
    if any(x in k for x in freeze):
        print(f'freezing {k}')
        v.requires_grad = False

# Optimizer
nbs = 64  # nominal batch size
accumulate = max(round(nbs / batch_size), 1)  # accumulate loss before optimizing
hyp['weight_decay'] *= batch_size * accumulate / nbs  # scale weight_decay
LOGGER.info(f"Scaled weight_decay = {hyp['weight_decay']}")

g0, g1, g2 = [], [], []  # optimizer parameter groups
for v in model.modules():
    if hasattr(v, 'bias') and isinstance(v.bias, nn.Parameter):  # bias
        g2.append(v.bias)
    if isinstance(v, nn.BatchNorm2d):  # weight (no decay)
        g0.append(v.weight)
    elif hasattr(v, 'weight') and isinstance(v.weight, nn.Parameter):  # weight (with decay)
        g1.append(v.weight)

if opt.adam:
    optimizer = Adam(g0, lr=hyp['lr0'], betas=(hyp['momentum'], 0.999))  # adjust beta1 to momentum
else:
    optimizer = SGD(g0, lr=hyp['lr0'], momentum=hyp['momentum'], nesterov=True)

optimizer.add_param_group({'params': g1, 'weight_decay': hyp['weight_decay']})  # add g1 with weight_decay
optimizer.add_param_group({'params': g2})  # add g2 (biases)
LOGGER.info(f"{colorstr('optimizer:')} {type(optimizer).__name__} with parameter groups "
            f"{len(g0)} weight, {len(g1)} weight (no decay), {len(g2)} bias")
del g0, g1, g2

# Scheduler
if opt.linear_lr:
    lf = lambda x: (1 - x / (epochs - 1)) * (1.0 - hyp['lrf']) + hyp['lrf']  # linear
else:
    lf = one_cycle(1, hyp['lrf'], epochs)  # cosine 1->hyp['lrf']
scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lf)  # plot_lr_scheduler(optimizer, scheduler, epochs)

# EMA
ema = ModelEMA(model) if RANK in [-1, 0] else None

# Resume
start_epoch, best_fitness = 0, 0.0
if pretrained:
    # Optimizer
    if ckpt['optimizer'] is not None:
        optimizer.load_state_dict(ckpt['optimizer'])
        best_fitness = ckpt['best_fitness']

    # EMA
    if ema and ckpt.get('ema'):
        ema.ema.load_state_dict(ckpt['ema'].float().state_dict())
        ema.updates = ckpt['updates']

    # Epochs
    start_epoch = ckpt['epoch'] + 1
    if resume:
        assert start_epoch > 0, f'{weights} training to {epochs} epochs is finished, nothing to resume.'
    if epochs < start_epoch:
        LOGGER.info(f"{weights} has been trained for {ckpt['epoch']} epochs. Fine-tuning for {epochs} more epochs.")
        epochs += ckpt['epoch']  # finetune additional epochs

    del ckpt, csd

# Image sizes
gs = max(int(model.stride.max()), 32)  # grid size (max stride)
nl = model.model[-1].nl  # number of detection layers (used for scaling hyp['obj'])
imgsz = check_img_size(opt.imgsz, gs, floor=gs * 2)  # verify imgsz is gs-multiple

# DP mode
if cuda and RANK == -1 and torch.cuda.device_count() > 1:
    logging.warning('DP not recommended, instead use torch.distributed.run for best DDP Multi-GPU results.\n'
                    'See Multi-GPU Tutorial at https://github.com/ultralytics/yolov5/issues/475 to get started.')
    model = torch.nn.DataParallel(model)

# SyncBatchNorm
if opt.sync_bn and cuda and RANK != -1:
    model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model).to(device)
    LOGGER.info('Using SyncBatchNorm()')

# Trainloader
train_loader, dataset = create_dataloader(train_path, imgsz, batch_size // WORLD_SIZE, gs, single_cls,
                                          hyp=hyp, augment=False, cache=opt.cache, rect=opt.rect, rank=RANK,
                                          workers=workers, image_weights=opt.image_weights, quad=opt.quad,
                                          prefix=colorstr('train: '))
n_grid_choices, n_anchor_choices = model.model[-1].nl, model.model[-1].na
grid_ratios = model.model[-1].stride.cpu().detach().numpy() / imgsz
# cstargets_all = read_crowdsourced_labels(data)
# cstargets_union = find_union_cstargets(cstargets_all['train'])
# cstargets_all_bcc = convert_cs_yolo2bcc(cstargets_all, n_anchor_choices, nc, grid_ratios)
# cstargets_bcc = torch.tensor(cstargets_all_bcc['train']) if torchMode else cstargets_all_bcc['train']
mlc = int(np.concatenate(dataset.labels, 0)[:, 0].max())  # max label class
nb = len(train_loader)  # number of batches
assert mlc < nc, f'Label class {mlc} exceeds nc={nc} in {data}. Possible class labels are 0-{nc - 1}'

# Process 0
if RANK in [-1, 0]:
    val_loader, val_dataset = create_dataloader(val_path, imgsz, batch_size // WORLD_SIZE * 2, gs, single_cls,
                                   hyp=hyp, cache=None if noval else opt.cache, rect=True, rank=-1,
                                   workers=workers, pad=0.5,
                                   prefix=colorstr('val: '))

    if not resume:
        labels = np.concatenate(dataset.labels, 0)
        if plots:
            plot_labels(labels, names, save_dir)

        # Anchors
        if not opt.noautoanchor:
            check_anchors(dataset, model=model, thr=hyp['anchor_t'], imgsz=imgsz)
        model.half().float()  # pre-reduce anchor precision

    callbacks.on_pretrain_routine_end()

# DDP mode
if cuda and RANK != -1:
    model = DDP(model, device_ids=[LOCAL_RANK], output_device=LOCAL_RANK)

# Model parameters
hyp['box'] *= 3. / nl  # scale to layers
hyp['cls'] *= nc / 80. * 3. / nl  # scale to classes and layers
hyp['obj'] *= (imgsz / 640) ** 2 * 3. / nl  # scale to image size and layers
hyp['label_smoothing'] = opt.label_smoothing
model.nc = nc  # attach number of classes to model
model.hyp = hyp  # attach hyperparameters to model
model.class_weights = labels_to_class_weights(dataset.labels, nc).to(device) * nc  # attach class weights
model.names = names

# Start training
t0 = time.time()
nw = max(round(hyp['warmup_epochs'] * nb), 1000)  # number of warmup iterations, max(3 epochs, 1k iterations)
last_opt_step = -1
maps = np.zeros(nc)  # mAP per class
results = (0, 0, 0, 0, 0, 0, 0)  # P, R, mAP@.5, mAP@.5-.95, val_loss(box, obj, cls)
scheduler.last_epoch = start_epoch - 1  # do not move
scaler = amp.GradScaler(enabled=cuda)
stopper = EarlyStopping(patience=opt.patience)
compute_loss = ComputeLoss(model)  # init loss class
# bcc_params = init_bcc_params(K=cstargets_bcc.shape[-1])
# bcc_params['n_epoch'] = epochs
# batch_pcm = {k: torch.tensor(v).to(device) if torchMode else v for k, v in compute_param_confusion_matrices(bcc_params).items()}
# # pred0_bcc = init_nn_output(dataset.n, grid_ratios, n_anchor_choices, bcc_params)
# bcc_metrics = init_metrics(bcc_params['n_epoch'])
LOGGER.info(f'Image sizes {imgsz} train, {imgsz} val\n'
            f'Using {train_loader.num_workers} dataloader workers\n'
            f"Logging results to {colorstr('bold', save_dir)}\n"
            f'Starting training for {epochs} epochs...')

times = defaultdict(float)
epoch_times = {epoch: defaultdict(float) for epoch in range(start_epoch, epochs)}
batch_times = {i: defaultdict(float) for i in range(1 + np.max(dataset.batch))}
epoch_batch_times = {epoch: {i: defaultdict(float) for i in batch_times} for epoch in epoch_times}
old_lb = float('Inf')

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.00043945312499999996, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.012500000000000002, cls_pw=10.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, mosaic=1.0, mixup=0.0, copy_paste=0.0, label_smoothing=0.0
TensorBoard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
Overriding model.yaml nc=80 with nc=2

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Focus                     [3, 32, 3]                    
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  

Plotting labels... 

autoanchor: Analyzing anchors... anchors/target = 6.06, Best Possible Recall (BPR) = 1.0000


Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/train/exp15
Starting training for 5 epochs...


In [62]:
targets

tensor([[ 0.00000,  0.00000,  0.10668,  0.27896,  0.05380,  0.05804],
        [ 0.00000,  0.00000,  0.30118,  0.26414,  0.06432,  0.05063],
        [ 0.00000,  0.00000,  0.45733,  0.28698,  0.05504,  0.05927],
        ...,
        [19.00000,  0.00000,  0.36910,  0.55416,  0.11159,  0.08688],
        [19.00000,  0.00000,  0.18830,  0.51502,  0.10193,  0.04505],
        [19.00000,  0.00000,  0.35730,  0.55792,  0.06652,  0.06221]])

In [126]:
[dataset.labels[i] for i in np.where(dataset.batch == b)[0]]

[array([[          0,     0.61749,     0.23584,     0.02897,     0.11756],
        [          0,     0.79399,     0.19051,    0.051502,    0.086402],
        [          0,     0.55794,     0.85127,    0.077253,    0.082153],
        [          0,     0.77039,     0.81232,    0.049356,    0.063739],
        [          0,     0.26663,     0.81516,    0.046137,    0.080737],
        [          1,     0.60408,     0.23584,     0.12232,     0.17139],
        [          1,      0.9485,     0.15297,    0.085837,     0.19263],
        [          0,     0.79185,     0.16572,    0.072961,    0.067989],
        [          0,      0.5574,     0.83286,     0.07618,    0.090652],
        [          0,     0.78326,     0.17989,    0.045064,     0.07932],
        [          0,     0.35622,     0.16431,    0.032189,    0.070822],
        [          0,     0.55472,     0.85057,    0.060086,    0.069405],
        [          0,     0.76609,     0.79603,    0.040773,     0.05949]], dtype=float32),
 array([

In [130]:
VOL_ID_MAP = {'Camellia': 0, 'Conghui': 1, 'HaoWen': 2, 'Xiongjie': 3}

def get_file_volunteers_dict(data_dict, mode='train', vol_id_map=VOL_ID_MAP):
    vol_path = os.path.join(data_dict['path'], 'volunteers')
    vol_mode_path = os.path.join(vol_path, mode)
    file_names = [x for x in os.listdir(vol_mode_path) if not x.startswith('.')]
    file_vols_dict = {}
    for fn in file_names:
        vol_file_path = os.path.join(vol_mode_path, fn)
        with open(vol_file_path) as f:
            vol_seq = [vol_id_map[x.strip()] for x in f.readlines()]
        file_vols_dict[fn] = torch.tensor(vol_seq).int()
    return file_vols_dict

In [125]:
sum([len(x) for x in [dataset.labels[i] for i in np.where(dataset.batch == b)[0]]])

239

In [144]:
file_volunteers_dict = get_file_volunteers_dict(data_dict)
b = 0
batch_filenames = [dataset.label_files[i].split(os.sep)[-1] for i in np.where(dataset.batch==b)[0]]
batch_volunteers_list = [file_volunteers_dict[fn] for fn in batch_filenames]
batch_volunteers = torch.cat(batch_volunteers_list)
target_volunteers = torch.cat([targets, batch_volunteers.unsqueeze(-1)], axis=1)
target_volunteers

tensor([[ 0.00000,  0.00000,  0.61749,  ...,  0.02897,  0.08891,  0.00000],
        [ 0.00000,  0.00000,  0.79399,  ...,  0.05150,  0.06534,  0.00000],
        [ 0.00000,  0.00000,  0.55794,  ...,  0.07725,  0.06213,  0.00000],
        ...,
        [19.00000,  0.00000,  0.79340,  ...,  0.06497,  0.03839,  2.00000],
        [19.00000,  0.00000,  0.81523,  ...,  0.06185,  0.06380,  3.00000],
        [19.00000,  0.00000,  0.78872,  ...,  0.05873,  0.04668,  3.00000]])

In [291]:
BACKGROUND_CLASS_ID

2

In [307]:
# Na, Nc, G, batch_size = n_anchor_choices, nc, grid_ratios, np.where(dataset.batch==b)[0].shape[0]
DEFAULT_G = np.array([1.0/32, 1.0/16, 1.0/8])


x = convert_target_volunteers_yolo2bcc(target_volunteers, n_anchor_choices, nc, grid_ratios, np.where(dataset.batch==b)[0].shape[0])
x.shape, x

(torch.Size([20, 25200, 4]),
 tensor([[[2., 2., 2., 2.],
          [2., 2., 2., 2.],
          [2., 2., 2., 2.],
          ...,
          [2., 2., 2., 2.],
          [2., 2., 2., 2.],
          [2., 2., 2., 2.]],
 
         [[2., 2., 2., 2.],
          [2., 2., 2., 2.],
          [2., 2., 2., 2.],
          ...,
          [2., 2., 2., 2.],
          [2., 2., 2., 2.],
          [2., 2., 2., 2.]],
 
         [[2., 2., 2., 2.],
          [2., 2., 2., 2.],
          [2., 2., 2., 2.],
          ...,
          [2., 2., 2., 2.],
          [2., 2., 2., 2.],
          [2., 2., 2., 2.]],
 
         ...,
 
         [[2., 2., 2., 2.],
          [2., 2., 2., 2.],
          [2., 2., 2., 2.],
          ...,
          [2., 2., 2., 2.],
          [2., 2., 2., 2.],
          [2., 2., 2., 2.]],
 
         [[2., 2., 2., 2.],
          [2., 2., 2., 2.],
          [2., 2., 2., 2.],
          ...,
          [2., 2., 2., 2.],
          [2., 2., 2., 2.],
          [2., 2., 2., 2.]],
 
         [[2., 2., 2., 2.

In [298]:
torch.cat((targets_per_ig_bcc, targets_per_ig_bcc)).shape

torch.Size([38400, 4])

In [289]:
Na

3

In [277]:
y = x.expand((2, x.shape[-1]))
y.shape, y

(torch.Size([2, 6400]),
 tensor([[2., 2., 2.,  ..., 2., 2., 2.],
         [2., 2., 2.,  ..., 2., 2., 2.]]))

In [252]:
targets_per_iv_bcc.stack(targets_per_iv_bcc)

AttributeError: 'Tensor' object has no attribute 'stack'

In [167]:
n_x_cells = n_y_cells = np.ceil(1/g_frac).astype(int)

2

In [164]:
g_frac = 0.5
find_containing_grid_cell(g_frac, g_frac, 0.25, 0.25)

0

In [127]:
train_loader, dataset = create_dataloader(train_path, imgsz, batch_size // WORLD_SIZE, gs, single_cls,
                                          hyp=hyp, augment=False, cache=opt.cache, rect=opt.rect, rank=RANK,
                                          workers=workers, image_weights=opt.image_weights, quad=opt.quad,
                                          prefix=colorstr('train: '))
# pbar = enumerate(train_loader)
# pbar = tqdm(pbar, total=nb)
for i, (imgs, targets, paths, _) in enumerate(train_loader):
    print(i, targets.shape)
    break

train: Scanning 'datasets/bcc/labels/train.cache' images and labels... 80 found, 0 missing, 4 empty, 0 corrupted: 100%|██████████| 80/80 [00:00<?, ?it/s]

0 torch.Size([239, 6])


In [12]:
DATA = {}
for epoch in range(start_epoch, epochs):  # epoch ------------------------------------------------------------------
    epoch_key = f'epoch_{epoch}'
    DATA[epoch_key] = {}
    bcc_flag = False if bcc_epoch == -1 else (epoch - start_epoch >= bcc_epoch)
    qtfilter_flag = False if qtfilter_epoch == -1 else (epoch - start_epoch >= qtfilter_epoch)
    if not bcc_flag and qtfilter_flag:
        qtfilter_flag = False
    LBs = []
    model.train()
    # Update image weights (optional, single-GPU only)
    if opt.image_weights:
        cw = model.class_weights.cpu().numpy() * (1 - maps) ** 2 / nc  # class weights
        iw = labels_to_image_weights(dataset.labels, nc=nc, class_weights=cw)  # image weights
        dataset.indices = random.choices(range(dataset.n), weights=iw, k=dataset.n)  # rand weighted idx

    # Update mosaic border (optional)

    mloss = torch.zeros(3, device=device)  # mean losses
    if RANK != -1:
        train_loader.sampler.set_epoch(epoch)
    pbar = enumerate(train_loader)
    LOGGER.info(('\n' + '%10s' * 7) % ('Epoch', 'gpu_mem', 'box', 'obj', 'cls', 'labels', 'img_size'))
    LOGGER.info('\n' + ('YOLO+BCC' if bcc_flag else 'Only YOLO'))
    if RANK in [-1, 0]:
        pbar = tqdm(pbar, total=nb)  # progress bar
    optimizer.zero_grad()

    for i, (imgs, targets, paths, _) in pbar:  # batch -------------------------------------------------------------
        batch_key = f'batch_{i}'
        DATA[epoch_key][batch_key] = {}
        batch_cstargets_bcc = (cstargets_bcc[dataset.batch == i]).to(device)
        ni = i + nb * epoch  # number integrated batches (since train start)
        DATA[epoch_key][batch_key]['imgs_unn'] = imgs.cpu().detach().numpy()
        DATA[epoch_key][batch_key]['targets'] = targets.cpu().detach().numpy()
        imgs = imgs.to(device, non_blocking=True).float() / 255.0  # uint8 to float32, 0-255 to 0.0-1.0

        # Warmup
        if ni <= nw:
            xi = [0, nw]  # x interp
            accumulate = max(1, np.interp(ni, xi, [1, nbs / batch_size]).round())
            for j, x in enumerate(optimizer.param_groups):
                # bias lr falls from 0.1 to lr0, all other lrs rise from 0.0 to lr0
                x['lr'] = np.interp(ni, xi, [hyp['warmup_bias_lr'] if j == 2 else 0.0, x['initial_lr'] * lf(epoch)])
                if 'momentum' in x:
                    x['momentum'] = np.interp(ni, xi, [hyp['warmup_momentum'], hyp['momentum']])

        # Multi-scale
        if opt.multi_scale:
            sz = random.randrange(imgsz * 0.5, imgsz * 1.5 + gs) // gs * gs  # size
            sf = sz / max(imgs.shape[2:])  # scale factor
            if sf != 1:
                ns = [math.ceil(x * sf / gs) * gs for x in imgs.shape[2:]]  # new shape (stretched to gs-multiple)
                imgs = nn.functional.interpolate(imgs, size=ns, mode='bilinear', align_corners=False)

        # Forward
        with amp.autocast(enabled=cuda):
            pred = model(imgs)
            DATA[epoch_key][batch_key]['pred'] = [temp.cpu().detach().numpy() for temp in pred]
            if bcc_flag:
                model.eval()
                
                batch_pred_yolo = nn_predict(model, imgs, imgsz, transform_format_flag=False) # y_hat_yolo
                DATA[epoch_key][batch_key]['batch_pred_yolo'] = batch_pred_yolo.cpu().detach().numpy()
                
                batch_pred_bcc, batch_pred_yolo_wh, batch_conf = yolo2bcc_newer(batch_pred_yolo, imgsz, silent=False) # y_hat_bcc
                DATA[epoch_key][batch_key]['batch_pred_bcc'] = batch_pred_bcc.cpu().detach().numpy()
                
                batch_qtargets, batch_pcm['variational'], batch_lb = VBi_yolo(batch_cstargets_bcc, batch_pred_bcc, batch_pcm['variational'], batch_pcm['prior'], torchMode = torchMode, device=device)
                DATA[epoch_key][batch_key]['batch_qtargets'] = batch_qtargets.cpu().detach().numpy()
                DATA[epoch_key][batch_key]['batch_pcm_var'] = batch_pcm['variational'].cpu().detach().numpy()
                DATA[epoch_key][batch_key]['batch_lb'] = batch_lb.cpu().detach().numpy()

                LBs.append(batch_lb)
                with torch.no_grad():
                    batch_qtargets_yolo = qt2yolo_optimized(batch_qtargets, grid_ratios, n_anchor_choices, batch_pred_yolo_wh, torchMode = torchMode, device=device).half().float()
                    DATA[epoch_key][batch_key]['batch_qtargets_yolo_full'] = batch_qtargets_yolo.cpu().detach().numpy()
                    batch_qtargets_yolo = batch_qtargets_yolo[batch_qtargets_yolo[:,1] != BACKGROUND_CLASS_ID, :]
                    DATA[epoch_key][batch_key]['batch_qtargets_yolo'] = batch_qtargets_yolo.cpu().detach().numpy()
                model.train()
                loss, loss_items = compute_loss(pred, batch_qtargets_yolo)
                DATA[epoch_key][batch_key]['loss'] = loss.cpu().detach().numpy()
                DATA[epoch_key][batch_key]['loss_items'] = loss_items.cpu().detach().numpy()
            else:
                # # # # # Just for the sake of seeing the output
                # model.eval()
                # batch_pred_yolo = nn_predict(model, imgs, imgsz, transform_format_flag=False) # y_hat_yolo
                # batch_pred_bcc, batch_pred_yolo_wh, batch_conf = yolo2bcc_newer(batch_pred_yolo, imgsz, silent=False) # y_hat_bcc
                # model.train()
                # # # # #
                batch_cstargets_union = targetize([cstargets_union[index] for index in np.where(dataset.batch == i)[0]])
                batch_cstargets_union = torch.cat(batch_cstargets_union)
                DATA[epoch_key][batch_key]['batch_cstargets_union'] = batch_cstargets_union.cpu().detach().numpy()
                loss, loss_items = compute_loss(pred, batch_cstargets_union.to(device))
#                 loss, loss_items = compute_loss(pred, targets.to(device))
                DATA[epoch_key][batch_key]['loss'] = loss.cpu().detach().numpy()
                DATA[epoch_key][batch_key]['loss_items'] = loss_items.cpu().detach().numpy()
            if RANK != -1:
                loss *= WORLD_SIZE  # gradient averaged between devices in DDP mode
                DATA[epoch_key][batch_key]['loss*WORLD_SIZE'] = loss.cpu().detach().numpy()
            if opt.quad:
                loss *= 4.
                DATA[epoch_key][batch_key]['loss*4'] = loss.cpu().detach().numpy()

        # Backward
        scaler.scale(loss).backward()

        # Optimize
        if ni - last_opt_step >= accumulate:
            scaler.step(optimizer)  # optimizer.step
            scaler.update()
            optimizer.zero_grad()
            if ema:
                ema.update(model)
            last_opt_step = ni

        # Log
        if RANK in [-1, 0]:
            mloss = (mloss * i + loss_items) / (i + 1)  # update mean losses
            DATA[epoch_key][batch_key]['mloss'] = mloss.cpu().detach().numpy()
            mem = f'{torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0:.3g}G'  # (GB)
            pbar.set_description(('%10s' * 2 + '%10.4g' * 5) % (
                f'{epoch}/{epochs - 1}', mem, *mloss, (batch_cstargets_union if bcc_flag else targets).shape[0], imgs.shape[-1]))
            callbacks.on_train_batch_end(ni, model, imgs, (batch_cstargets_union if bcc_flag else targets), paths, plots, opt.sync_bn)
        # end batch ------------------------------------------------------------------------------------------------

    # Scheduler
    lr = [x['lr'] for x in optimizer.param_groups]  # for loggers
    
    scheduler.step()

    if RANK in [-1, 0]:
        # mAP
        callbacks.on_train_epoch_end(epoch=epoch)
        ema.update_attr(model, include=['yaml', 'nc', 'hyp', 'names', 'stride', 'class_weights'])
        final_epoch = epoch + 1 == epochs
        if not noval or final_epoch:  # Calculate mAP
            results, maps, _ = val.run(data_dict,
                                       batch_size=batch_size // WORLD_SIZE * 2,
                                       imgsz=imgsz,
                                       model=ema.ema,
                                       single_cls=single_cls,
                                       dataloader=val_loader,
                                       save_dir=save_dir,
                                       save_json=is_coco and final_epoch,
                                       verbose=nc < 50 and final_epoch,
                                       plots=plots and final_epoch,
                                       callbacks=callbacks,
                                       compute_loss=compute_loss)
            # yhat_train = pred
            # y_train = dataset.labels
            # yhat_test = results
            # y_test = val_dataset.labels
            # update_bcc_metrics(bcc_metrics, qtargets, yhat_train, y_train, yhat_test, y_test, epoch)

        # Update best mAP
        fi = fitness(np.array(results).reshape(1, -1))  # weighted combination of [P, R, mAP@.5, mAP@.5-.95]
        if fi > best_fitness:
            best_fitness = fi
        log_vals = list(mloss) + list(results) + lr
        callbacks.on_fit_epoch_end(log_vals, epoch, best_fitness, fi)

        # Save model
        if (not nosave) or (final_epoch and not evolve):  # if save
            ckpt = {'epoch': epoch,
                    'best_fitness': best_fitness,
                    'model': deepcopy(de_parallel(model)).half(),
                    'ema': deepcopy(ema.ema).half(),
                    'updates': ema.updates,
                    'optimizer': optimizer.state_dict(),
                    'wandb_id': loggers.wandb.wandb_run.id if loggers.wandb else None}

            # Save last, best and delete
            torch.save(ckpt, last)
            if best_fitness == fi:
                torch.save(ckpt, best)
            del ckpt
            callbacks.on_model_save(last, epoch, final_epoch, best_fitness, fi)

        # Stop Single-GPU
        if stopper(epoch=epoch, fitness=fi):
            break
    try:
        lb = LBs[-1]
        if epoch > start_epoch and torch.abs((lb - old_lb) / old_lb) < bcc_params['convergence_threshold']:
            print('Convergence reached!')
            break
        old_lb = lb
    except IndexError:
        pass

    try:

        del batch_pred_yolo, batch_pred_bcc, batch_pred_yolo_wh, batch_qtargets, batch_qtargets_yolo
    except (UnboundLocalError, NameError):
        pass


#     torch.cuda.empty_cache()

    # end epoch ----------------------------------------------------------------------------------------------------

# end training -----------------------------------------------------------------------------------------------------


     Epoch   gpu_mem       box       obj       cls    labels  img_size

YOLO+BCC
  0%|          | 0/1 [00:00<?, ?it/s]

Minimum probs (c1, c2, bkgd): [2.3e-05, 1e-05, 0.927898]
Maximum probs (c1, c2, bkgd): [0.032962, 0.04197, 0.999958]


       0/4        0G         0   0.02347         0         4       640: 100%|██████████| 1/1 [00:11<00:00, 11.16s/it]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100%|██████████| 1/1 [00:01<00:00,  1.56s/it]


                 all          2          0          0          0          0          0



     Epoch   gpu_mem       box       obj       cls    labels  img_size

YOLO+BCC
  0%|          | 0/1 [00:00<?, ?it/s]

...

                 all          2          6     0.0171        0.1    0.00416   0.000416



     Epoch   gpu_mem       box       obj       cls    labels  img_size

YOLO+BCC
  0%|          | 0/1 [00:00<?, ?it/s]

Minimum probs (c1, c2, bkgd): [2.3e-05, 1e-06, 0.890421]
Maximum probs (c1, c2, bkgd): [0.050024, 0.059555, 0.999967]


       4/4        0G    0.1218   0.03394    0.1979         4       640: 100%|██████████| 1/1 [00:02<00:00,  2.62s/it]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100%|██████████| 1/1 [00:01<00:00,  1.38s/it]


                 all          2          6      0.016        0.1    0.00431   0.000431
           bone-loss          2          5     0.0321        0.2    0.00862   0.000862
       dental-caries          2          1          0          0          0          0


In [13]:
if bcc_epoch == -1:
    YOLO_DATA = DATA
elif bcc_epoch == 0:
    YOLOBCC_DATA = DATA

In [14]:
YOLOBCC_DATA == YOLO_DATA

False

In [15]:
from label_converter import equals

In [16]:
equals(YOLOBCC_DATA, YOLO_DATA)

False

In [24]:
import pandas as pd
df1 = pd.DataFrame({k: v['batch_0'] for k, v in YOLOBCC_DATA.items()})
df2 = pd.DataFrame({k: v['batch_0'] for k, v in YOLO_DATA.items()})

In [25]:
df1

,epoch_0,epoch_1,epoch_2,epoch_3,epoch_4
imgs_unn,[[[[114 114 114 114 114 114 114 114 114 114 11...,[[[[114 114 114 114 114 114 114 114 114 114 11...,[[[[114 114 114 114 114 114 114 114 114 114 11...,[[[[114 114 114 114 114 114 114 114 114 114 11...,[[[[114 114 114 114 114 114 114 114 114 114 11...
targets,"[[0.0, 1.0, 0.52132195, 0.34869355, 0.31544566...","[[0.0, 1.0, 0.52132195, 0.34869355, 0.31544566...","[[0.0, 1.0, 0.52132195, 0.34869355, 0.31544566...","[[0.0, 1.0, 0.52132195, 0.34869355, 0.31544566...","[[0.0, 1.0, 0.52132195, 0.34869355, 0.31544566..."
pred,[[[[[[ -1.7136 2.8824 0.15735 ... ...,[[[[[[ -1.7136 2.8824 0.15742 ... ...,[[[[[[ -1.7139 2.8822 0.15708 ... ...,[[[[[[ -1.7071 2.9159 0.25172 ... ...,[[[[[[ -1.6883 2.894 0.43994 ... ...
batch_pred_yolo,"[[[0.0036954307, 0.009272168, 13.49456, 3.2351...","[[[0.009250021, 0.015687559, 8.035716, 1.97089...","[[[0.009267814, 0.015761245, 7.892433, 1.93131...","[[[0.00927645, 0.015868383, 7.665674, 1.833384...","[[[0.009246418, 0.015960094, 7.4783363, 1.8088..."
batch_pred_bcc,"[[[-6.57176, -9.356627, -0.0014868102], [-6.33...","[[[-8.388549, -11.618019, -0.00023647958], [-8...","[[[-8.388947, -11.647085, -0.00023612188], [-8...","[[[-8.375712, -11.674401, -0.00023892395], [-8...","[[[-8.385114, -11.707643, -0.00023647958], [-8..."
batch_qtargets,"[[[0.0012005094, 7.411618e-05, 0.99872535], [0...","[[[0.00022215437, 8.685021e-06, 0.99976915], [...","[[[0.00021797261, 8.190652e-06, 0.9997738], [0...","[[[0.00021654363, 7.657322e-06, 0.9997758], [0...","[[[0.00021121095, 7.137267e-06, 0.99978167], [..."
batch_pcm_var,"[[[1.1613655857741834], [1.1118390411138535], ...","[[[2.2191656589508058], [2.1804468631744385], ...","[[[3.1584545135498048], [3.199611186981201], [...","[[[3.895907497406006], [3.936058759689331], [9...","[[[4.4509540557861325], [4.347512245178223], [..."
batch_lb,-5034.257,-5728.213,-5747.6733,-5745.882,-5720.651
batch_qtargets_yolo_full,"[[0.0, 2.0, 0.006248474, 0.006248474, 0.021087...","[[0.0, 2.0, 0.006248474, 0.006248474, 0.012557...","[[0.0, 2.0, 0.006248474, 0.006248474, 0.012329...","[[0.0, 2.0, 0.006248474, 0.006248474, 0.011978...","[[0.0, 2.0, 0.006248474, 0.006248474, 0.011688..."
batch_qtargets_yolo,[],[],"[[1.0, 1.0, 0.72509766, 0.22497559, 0.16052246...","[[0.0, 1.0, 0.52490234, 0.32495117, 0.20776367...","[[0.0, 1.0, 0.52490234, 0.32495117, 0.2084961,..."


In [26]:
df2

,epoch_0,epoch_1,epoch_2,epoch_3,epoch_4
imgs_unn,[[[[114 114 114 114 114 114 114 114 114 114 11...,[[[[114 114 114 114 114 114 114 114 114 114 11...,[[[[114 114 114 114 114 114 114 114 114 114 11...,[[[[114 114 114 114 114 114 114 114 114 114 11...,[[[[114 114 114 114 114 114 114 114 114 114 11...
targets,"[[0.0, 1.0, 0.52132195, 0.34869355, 0.31544566...","[[0.0, 1.0, 0.52132195, 0.34869355, 0.31544566...","[[0.0, 1.0, 0.52132195, 0.34869355, 0.31544566...","[[0.0, 1.0, 0.52132195, 0.34869355, 0.31544566...","[[0.0, 1.0, 0.52132195, 0.34869355, 0.31544566..."
pred,[[[[[[ -1.7136 2.8824 0.15735 ... ...,[[[[[[ -1.7121 2.8829 0.15691 ... ...,[[[[[[ -1.7091 2.8879 0.16855 ... ...,[[[[[[ -1.706 2.8965 0.19091 ... ...,[[[[[[ -1.7032 2.9038 0.21242 ... ...
batch_cstargets_union,"[[0.0, 1.0, 0.5213219616204691, 0.302777777777...","[[0.0, 1.0, 0.5213219616204691, 0.302777777777...","[[0.0, 1.0, 0.5213219616204691, 0.302777777777...","[[0.0, 1.0, 0.5213219616204691, 0.302777777777...","[[0.0, 1.0, 0.5213219616204691, 0.302777777777..."
loss,[0.4964128],[0.49444667],[0.49096644],[0.48612297],[0.48050168]
loss_items,"[0.07071755, 0.033150956, 0.1443379]","[0.070573285, 0.0331745, 0.14347555]","[0.07040738, 0.033200026, 0.14187582]","[0.070230246, 0.033224806, 0.13960643]","[0.070089616, 0.03323116, 0.13693006]"
mloss,"[0.07071755, 0.033150956, 0.1443379]","[0.070573285, 0.0331745, 0.14347555]","[0.07040738, 0.033200026, 0.14187582]","[0.070230246, 0.033224806, 0.13960643]","[0.070089616, 0.03323116, 0.13693006]"


In [56]:
def is_entry_eq(df1, df2, row1, col1, row2=None, col2=None, verbosity=1):
    if row2 is None:
        row2 = row1
    if col2 is None:
        col2 = col1
    flag = equals(df1.loc[row1, col1], df2.loc[row2, col2])
    if verbosity == 1:
        print(flag)
    rc1 = (row1, col1)
    rc2 = (row2, col2) if (row1 != row2 and col1 != col2) else (col1 if col1 != col2 else (row1 if row1 != row2 else ''))
    if verbosity == 2:
        print((row1, col1), (row2, col2), flag)

In [58]:
# Direct equality
rows = ['imgs_unn', 'targets', 'pred', 'loss', 'loss_items', 'mloss']
cols = [f'epoch_{i}' for i in range(5)]
for row in rows:
#     print('***', row, '***')
    for col in cols:
        is_entry_eq(df1, df2, row, col, verbosity=2)

('imgs_unn', 'epoch_0') ('imgs_unn', 'epoch_0') True
('imgs_unn', 'epoch_1') ('imgs_unn', 'epoch_1') True
('imgs_unn', 'epoch_2') ('imgs_unn', 'epoch_2') True
('imgs_unn', 'epoch_3') ('imgs_unn', 'epoch_3') True
('imgs_unn', 'epoch_4') ('imgs_unn', 'epoch_4') True
('targets', 'epoch_0') ('targets', 'epoch_0') True
('targets', 'epoch_1') ('targets', 'epoch_1') True
('targets', 'epoch_2') ('targets', 'epoch_2') True
('targets', 'epoch_3') ('targets', 'epoch_3') True
('targets', 'epoch_4') ('targets', 'epoch_4') True
('pred', 'epoch_0') ('pred', 'epoch_0') True
('pred', 'epoch_1') ('pred', 'epoch_1') False
('pred', 'epoch_2') ('pred', 'epoch_2') False
('pred', 'epoch_3') ('pred', 'epoch_3') False
('pred', 'epoch_4') ('pred', 'epoch_4') False
('loss', 'epoch_0') ('loss', 'epoch_0') False
('loss', 'epoch_1') ('loss', 'epoch_1') False
('loss', 'epoch_2') ('loss', 'epoch_2') False
('loss', 'epoch_3') ('loss', 'epoch_3') False
('loss', 'epoch_4') ('loss', 'epoch_4') False
('loss_items', 'epoch

In [59]:
# Equality-pairs
row_pairs = [('targets', 'batch_cstargets_union')]
for row1, row2 in row_pairs:
    for col in cols:
        is_entry_eq(df1, df2, row1, col, row2, col, verbosity=2)

('targets', 'epoch_0') ('batch_cstargets_union', 'epoch_0') False
('targets', 'epoch_1') ('batch_cstargets_union', 'epoch_1') False
('targets', 'epoch_2') ('batch_cstargets_union', 'epoch_2') False
('targets', 'epoch_3') ('batch_cstargets_union', 'epoch_3') False
('targets', 'epoch_4') ('batch_cstargets_union', 'epoch_4') False


In [62]:
df1.loc['targets', 'epoch_0'], df2.loc['batch_cstargets_union', 'epoch_0']

(array([[          0,           1,     0.52132,     0.34869,     0.31545,     0.31528],
        [          1,           1,     0.71672,     0.27838,     0.10863,     0.13925],
        [          1,           1,     0.66294,     0.60045,     0.10543,     0.13074],
        [          1,           0,     0.18477,     0.58291,     0.13099,     0.11905]], dtype=float32),
 array([[          0,           1,     0.52132,     0.30278,     0.31545,     0.41096],
        [          1,           1,     0.71672,     0.20634,     0.10863,     0.18451],
        [          1,           1,     0.66294,      0.6331,     0.10543,     0.17324],
        [          1,           0,     0.18477,     0.60986,     0.13099,     0.15775]]))

In [93]:
# def read_crowdsourced_labels(data):
EXPERT_NAME = 'Jonathan'
if isinstance(data, str):
    data_dict = check_dataset(data)
elif isinstance(data, dict):
    data_dict = data
cs_root_path = os.path.join('{}_crowdsourced'.format(data_dict['path']), 'labels')
volunteers_root_path = os.path.join(data_dict['path'], 'volunteers')
if not os.path.isdir(volunteers_root_path):
    os.mkdir(volunteers_root_path)
volunteers = list(sorted([x for x in os.listdir(cs_root_path) if (not x.startswith('.') and x != 'all')]))
users = [EXPERT_NAME] + volunteers

user_id_map = {u: i for i, u in enumerate(users)}
id_user_map = {i: u for u, i in user_id_map.items()}


modes = ['train', 'test', 'val']
y_crowdsourced = {m: [] for m in modes}
for m in modes:
    image_file_prefixes = [os.path.splitext(n)[0] for n in \
    os.listdir(os.path.join(data_dict['path'], 'images', m)) \
        if not n.startswith('.')]
    y_per_mode = []
    for u in users:
        user_id = user_id_map[u]
        y_per_mode_per_user = []
        user_path = os.path.join(cs_root_path, u)
        if not os.path.exists(user_path):
            continue
        path = , m)
        for file_prefix in image_file_prefixes:
            label_file_name = f'{file_prefix}.txt'
            label_file_path = os.path.join(path, label_file_name)
            try:
                img_labels = np.loadtxt(label_file_path)
            except OSError:
                img_labels = np.array([-1, 0, 0, 0, 0])
            if img_labels.ndim == 1:
                img_labels = np.expand_dims(img_labels, axis = 0)
                user_id_seq = [user_id]*img_labels.shape[0]
#             user_img_labels = np.concatenate([np.ones((img_labels.shape[0], 1))*user_id, img_labels], axis=1)
            # with open(os.path.join(path, file_name), 'r') as f:
            #     img_labels = f.readlines()
            y_per_mode_per_user.append(img_labels)
        y_per_mode.append(y_per_mode_per_user)
    y_crowdsourced[m] = y_per_mode
#     return y_crowdsourced


## Data Preparation

In [114]:
!tree datasets/master

datasets/master
├── images
│   ├── IS20180614_161235_0835_000000CD.jpg
│   ├── IS20190121_094120_0903_00000702.jpg
│   ├── IS20190529_121316_0603_00001B5F.jpg
│   ├── IS20190605_100053_0811_00001CCE.jpg
│   ├── IS20190716_144826_0668_000026F2.jpg
│   ├── IS20190716_144902_0220_000026F3.jpg
│   ├── IS20190916_155531_0888_00003368.jpg
│   ├── IS20190925_080311_0732_0000343E.jpg
│   ├── IS20190930_080551_0119_000034B9.jpg
│   ├── IS20191111_161011_0377_0000387B.jpg
│   ├── IS20191113_081626_0893_000038CF.jpg
│   ├── IS20191121_115147_0636_000039A3.jpg
│   ├── IS20191125_105634_0062_000039C9.jpg
│   ├── IS20191128_142111_0361_00003A3B.jpg
│   ├── IX20191008_105239_0006_00003581.jpg
│   ├── IX20191018_145924_0858_000036ED.jpg
│   ├── IX20191022_101757_0873_00003708.jpg
│   ├── IX20191023_142706_0883_0000373E.jpg
│   ├── IX20191024_142538_0552_00003761.jpg
│   ├── IX20191030_093817_0791_000037B5.jpg
│   ├── Unknown-X-20191111-111540-XE2TAIKRCCFW-3.jpg
│   ├── Unknown-X-20191113-094338-X9SZC0

#### Read Master Data

#### Convert to IID Data

Exception: Path already exists. Not recreating data

#### Convert to BCC Data

Exception: Path already exists. Not recreating data.

In [7]:
df[df[0] == ]

30      1.0
76      1.0
21      1.0
46      1.0
75      2.0
      ...  
73     29.0
56     35.0
41     36.0
2      46.0
0     254.0
Length: 71, dtype: float64

In [291]:
temp = []
for i in range(80):
    for j in range(80):
        if i == j:
            continue
        try:
            c = df[(df[i]>0) & (df[j]>0)].shape[0]
            print(i, j, c)
            temp.append((c, i, j))
        except KeyError:
            print(i, j, c)

0 1 3
0 2 4
0 3 2
0 4 1
0 5 4
0 6 1
0 7 1
0 8 0
0 9 2
0 10 2
0 11 1
0 12 1
0 13 5
0 14 1
0 15 1
0 16 2
0 17 1
0 18 1
0 19 1
0 20 1
0 21 0
0 22 0
0 23 1
0 24 4
0 25 4
0 26 9
0 27 6
0 28 2
0 29 3
0 30 1
0 31 2
0 32 6
0 33 2
0 34 4
0 35 4
0 36 2
0 37 2
0 38 5
0 39 1
0 40 2
0 41 5
0 42 3
0 43 4
0 44 2
0 45 0
0 46 0
0 47 0
0 48 0
0 49 0
0 50 1
0 51 0
0 52 0
0 53 2
0 54 1
0 55 3
0 56 7
0 57 3
0 58 4
0 59 0
0 60 4
0 61 0
0 62 0
0 63 0
0 64 0
0 65 3
0 66 3
0 67 3
0 68 1
0 69 1
0 70 1
0 71 0
0 72 1
0 73 3
0 74 3
0 75 0
0 76 1
0 77 1
0 78 1
0 79 0
1 0 3
1 2 0
1 3 0
1 4 0
1 5 1
1 6 0
1 7 0
1 8 0
1 9 0
1 10 0
1 11 0
1 12 0
1 13 0
1 14 0
1 15 0
1 16 1
1 17 0
1 18 0
1 19 0
1 20 0
1 21 0
1 22 0
1 23 0
1 24 0
1 25 0
1 26 0
1 27 0
1 28 0
1 29 0
1 30 0
1 31 0
1 32 1
1 33 0
1 34 0
1 35 0
1 36 0
1 37 0
1 38 1
1 39 0
1 40 0
1 41 0
1 42 0
1 43 0
1 44 0
1 45 0
1 46 0
1 47 0
1 48 0
1 49 0
1 50 0
1 51 0
1 52 0
1 53 0
1 54 0
1 55 0
1 56 0
1 57 0
1 58 0
1 59 0
1 60 0
1 61 0
1 62 0
1 63 0
1 64 0
1 65 0
1 66 0
1 6

15 52 0
15 53 0
15 54 0
15 55 0
15 56 0
15 57 2
15 58 1
15 59 0
15 60 0
15 61 0
15 62 0
15 63 0
15 64 0
15 65 2
15 66 2
15 67 0
15 68 0
15 69 0
15 70 0
15 71 0
15 72 0
15 73 0
15 74 0
15 75 0
15 76 0
15 77 0
15 78 0
15 79 0
16 0 2
16 1 1
16 2 1
16 3 0
16 4 0
16 5 0
16 6 0
16 7 0
16 8 1
16 9 0
16 10 0
16 11 0
16 12 0
16 13 1
16 14 0
16 15 0
16 17 0
16 18 0
16 19 0
16 20 0
16 21 0
16 22 0
16 23 0
16 24 0
16 25 0
16 26 0
16 27 0
16 28 0
16 29 2
16 30 0
16 31 0
16 32 0
16 33 0
16 34 0
16 35 0
16 36 1
16 37 1
16 38 0
16 39 0
16 40 0
16 41 0
16 42 0
16 43 0
16 44 0
16 45 0
16 46 0
16 47 0
16 48 0
16 49 0
16 50 0
16 51 0
16 52 0
16 53 0
16 54 0
16 55 0
16 56 0
16 57 0
16 58 0
16 59 0
16 60 0
16 61 0
16 62 0
16 63 0
16 64 0
16 65 0
16 66 0
16 67 0
16 68 0
16 69 0
16 70 0
16 71 0
16 72 0
16 73 0
16 74 0
16 75 0
16 76 0
16 77 1
16 78 1
16 79 0
17 0 1
17 1 0
17 2 0
17 3 0
17 4 0
17 5 0
17 6 0
17 7 0
17 8 0
17 9 0
17 10 0
17 11 0
17 12 0
17 13 0
17 14 0
17 15 0
17 16 0
17 18 0
17 19 0
17 20 0
17 2

29 22 0
29 23 0
29 24 0
29 25 0
29 26 0
29 27 0
29 28 0
29 30 0
29 31 0
29 32 0
29 33 0
29 34 0
29 35 0
29 36 0
29 37 0
29 38 0
29 39 0
29 40 0
29 41 0
29 42 0
29 43 0
29 44 0
29 45 0
29 46 0
29 47 0
29 48 0
29 49 0
29 50 0
29 51 0
29 52 0
29 53 0
29 54 0
29 55 0
29 56 0
29 57 0
29 58 0
29 59 0
29 60 0
29 61 0
29 62 0
29 63 0
29 64 0
29 65 0
29 66 0
29 67 0
29 68 0
29 69 0
29 70 0
29 71 0
29 72 0
29 73 0
29 74 0
29 75 0
29 76 0
29 77 0
29 78 0
29 79 0
30 0 1
30 1 0
30 2 0
30 3 0
30 4 0
30 5 0
30 6 0
30 7 0
30 8 0
30 9 0
30 10 0
30 11 0
30 12 0
30 13 0
30 14 0
30 15 0
30 16 0
30 17 0
30 18 0
30 19 0
30 20 0
30 21 0
30 22 0
30 23 0
30 24 1
30 25 0
30 26 0
30 27 0
30 28 0
30 29 0
30 31 0
30 32 0
30 33 0
30 34 0
30 35 0
30 36 0
30 37 0
30 38 0
30 39 0
30 40 0
30 41 0
30 42 0
30 43 0
30 44 0
30 45 0
30 46 0
30 47 0
30 48 0
30 49 0
30 50 0
30 51 0
30 52 0
30 53 0
30 54 0
30 55 0
30 56 0
30 57 0
30 58 0
30 59 0
30 60 0
30 61 0
30 62 0
30 63 0
30 64 0
30 65 0
30 66 0
30 67 0
30 68 0
30 69 0
30

43 6 0
43 7 0
43 8 0
43 9 0
43 10 0
43 11 0
43 12 0
43 13 1
43 14 0
43 15 0
43 16 0
43 17 0
43 18 0
43 19 0
43 20 0
43 21 0
43 22 0
43 23 0
43 24 0
43 25 1
43 26 1
43 27 0
43 28 0
43 29 0
43 30 0
43 31 0
43 32 0
43 33 0
43 34 0
43 35 0
43 36 0
43 37 0
43 38 0
43 39 1
43 40 0
43 41 3
43 42 2
43 44 2
43 45 1
43 46 0
43 47 0
43 48 0
43 49 0
43 50 0
43 51 0
43 52 0
43 53 2
43 54 0
43 55 2
43 56 2
43 57 0
43 58 1
43 59 0
43 60 3
43 61 0
43 62 0
43 63 0
43 64 0
43 65 0
43 66 0
43 67 0
43 68 2
43 69 2
43 70 2
43 71 0
43 72 1
43 73 2
43 74 1
43 75 0
43 76 0
43 77 0
43 78 0
43 79 0
44 0 2
44 1 0
44 2 0
44 3 0
44 4 0
44 5 0
44 6 0
44 7 0
44 8 0
44 9 0
44 10 0
44 11 0
44 12 0
44 13 1
44 14 0
44 15 0
44 16 0
44 17 0
44 18 0
44 19 0
44 20 0
44 21 0
44 22 0
44 23 0
44 24 0
44 25 1
44 26 1
44 27 0
44 28 0
44 29 0
44 30 0
44 31 0
44 32 0
44 33 0
44 34 0
44 35 0
44 36 0
44 37 0
44 38 0
44 39 0
44 40 0
44 41 2
44 42 2
44 43 2
44 45 2
44 46 0
44 47 0
44 48 0
44 49 0
44 50 2
44 51 2
44 52 0
44 53 0
44 54 

57 62 0
57 63 1
57 64 1
57 65 5
57 66 5
57 67 0
57 68 0
57 69 0
57 70 0
57 71 0
57 72 0
57 73 2
57 74 1
57 75 0
57 76 0
57 77 0
57 78 0
57 79 0
58 0 4
58 1 0
58 2 0
58 3 0
58 4 0
58 5 0
58 6 1
58 7 0
58 8 0
58 9 0
58 10 0
58 11 0
58 12 0
58 13 1
58 14 0
58 15 1
58 16 0
58 17 1
58 18 1
58 19 1
58 20 0
58 21 0
58 22 0
58 23 0
58 24 0
58 25 1
58 26 2
58 27 0
58 28 0
58 29 0
58 30 0
58 31 0
58 32 0
58 33 0
58 34 0
58 35 0
58 36 0
58 37 0
58 38 0
58 39 1
58 40 0
58 41 2
58 42 0
58 43 1
58 44 1
58 45 1
58 46 0
58 47 0
58 48 0
58 49 0
58 50 0
58 51 0
58 52 0
58 53 0
58 54 0
58 55 1
58 56 0
58 57 3
58 59 0
58 60 1
58 61 1
58 62 0
58 63 0
58 64 0
58 65 3
58 66 3
58 67 1
58 68 0
58 69 1
58 70 1
58 71 2
58 72 1
58 73 2
58 74 1
58 75 2
58 76 0
58 77 0
58 78 0
58 79 0
59 0 0
59 1 0
59 2 0
59 3 0
59 4 0
59 5 0
59 6 0
59 7 0
59 8 0
59 9 0
59 10 0
59 11 0
59 12 0
59 13 0
59 14 0
59 15 0
59 16 0
59 17 0
59 18 0
59 19 0
59 20 0
59 21 0
59 22 0
59 23 0
59 24 0
59 25 0
59 26 0
59 27 0
59 28 0
59 29 0
59 3

72 23 0
72 24 0
72 25 0
72 26 1
72 27 0
72 28 0
72 29 0
72 30 0
72 31 0
72 32 0
72 33 0
72 34 0
72 35 0
72 36 0
72 37 0
72 38 0
72 39 2
72 40 1
72 41 1
72 42 0
72 43 1
72 44 0
72 45 3
72 46 0
72 47 0
72 48 0
72 49 0
72 50 0
72 51 0
72 52 0
72 53 1
72 54 0
72 55 0
72 56 1
72 57 0
72 58 1
72 59 0
72 60 1
72 61 0
72 62 0
72 63 0
72 64 0
72 65 0
72 66 0
72 67 0
72 68 1
72 69 4
72 70 4
72 71 1
72 73 0
72 74 2
72 75 1
72 76 0
72 77 0
72 78 0
72 79 0
73 0 3
73 1 0
73 2 1
73 3 0
73 4 0
73 5 0
73 6 0
73 7 0
73 8 0
73 9 0
73 10 0
73 11 0
73 12 0
73 13 2
73 14 0
73 15 0
73 16 0
73 17 0
73 18 0
73 19 0
73 20 0
73 21 0
73 22 0
73 23 0
73 24 1
73 25 1
73 26 1
73 27 1
73 28 0
73 29 0
73 30 0
73 31 0
73 32 0
73 33 0
73 34 0
73 35 0
73 36 0
73 37 0
73 38 0
73 39 1
73 40 0
73 41 1
73 42 0
73 43 2
73 44 1
73 45 1
73 46 0
73 47 0
73 48 0
73 49 0
73 50 0
73 51 0
73 52 0
73 53 0
73 54 0
73 55 1
73 56 0
73 57 2
73 58 2
73 59 0
73 60 1
73 61 0
73 62 1
73 63 1
73 64 2
73 65 2
73 66 2
73 67 1
73 68 1
73 69 1
73